In [ ]:
#-------------------------| Vacuum Surface Energy |----------------------------------#
#------------------------------| In eV/ang^2 |---------------------------------------#
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from pathlib import Path
from ase.io import read

parent_directory = Path("/Users/bdayers/Downloads/Surface-energy")
N_values = np.array([1, 3, 5, 7, 9, 11, 13, 15])
Eslab_values = []
A_value = None

for subdirectory in parent_directory.rglob("*-thick"):
    if subdirectory.is_dir():
        out_file = next(subdirectory.glob("Li.out"), None)
        if out_file:
            atoms = read(str(out_file))
            energy = atoms.get_potential_energy()
            Eslab_values.append(energy)
            Eslab_values = np.array(Eslab_values)
            np.array(Eslab_values)
            if A_value is None:
                A_value = np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1]))

Eslab_values = np.sort(Eslab_values)[::-1]
surface_energy_dict = {}
plt.figure(figsize=(8, 6))

for i in range(len(N_values) - 1):
    slope, _, _, _, _ = linregress(N_values[i:], Eslab_values[i:])
    surface_energy = (1 / (2 * A_value)) * (Eslab_values - (N_values * slope))
    key = f"{N_values[i]} to {N_values[-1]}"
    surface_energy_dict[key] = surface_energy
    plt.plot(range(len(N_values)), surface_energy, marker='o', label=key)

plt.xlabel('Number of Layers')
plt.ylabel('Surface Energy (eV $Å^{-2}$)')
plt.xticks(range(len(N_values)), N_values)
plt.legend()
plt.grid(alpha=0.5, linestyle='--')
plt.title('Surface Energy vs N for Different Ranges')
plt.show()

chosen_key = input("Enter the key of the chosen range: ")

plt.figure(figsize=(8, 6))
chosen_surface_energy = surface_energy_dict[chosen_key]
print(chosen_surface_energy)
plt.plot(range(len(N_values)), chosen_surface_energy, marker='o', label="Fiorentini Method")
plt.xlabel('Number of Layers')
plt.ylabel('Surface Energy (eV $Å^{-2}$)')
plt.xticks(range(len(N_values)), N_values)
plt.legend()
plt.grid(alpha=0.7, linestyle='--')
plt.savefig("Me_3.440-in-ev.pdf", dpi=700)
plt.show()


In [ ]:
#-------------------------| Vacuum Surface Energy |----------------------------------#
#-------------------------------| In J/m^2 |-----------------------------------------#
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from pathlib import Path
from ase.io import read

parent_directory = Path("/Users/bdayers/Downloads/Surface-energy")
N_values = np.array([1, 3, 5, 7, 9, 11, 13, 15])
Eslab_values = []
A_value = None
ev_to_joules = 1.602176634E-19
for subdirectory in parent_directory.rglob("*-thick"):
    if subdirectory.is_dir():
        out_file = next(subdirectory.glob("Li.out"), None)
        if out_file:
            atoms = read(str(out_file))
            energy = atoms.get_potential_energy() * ev_to_joules
            Eslab_values.append(energy)
            Eslab_values = np.array(Eslab_values)
            np.array(Eslab_values)
            if A_value is None:
                A_value = np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1])) * 1E-20

Eslab_values = np.sort(Eslab_values)[::-1]
surface_energy_dict = {}
plt.figure(figsize=(8, 6))

for i in range(len(N_values) - 1):
    slope, _, _, _, _ = linregress(N_values[i:], Eslab_values[i:])
    surface_energy = (1 / (2 * A_value)) * (Eslab_values - (N_values * slope))
    key = f"{N_values[i]} to {N_values[-1]}"
    surface_energy_dict[key] = surface_energy
    plt.plot(range(len(N_values)), surface_energy, marker='o', label=key)

plt.xlabel('Number of Layers')
plt.ylabel('Surface Energy (J $m^{-2}$)')
plt.xticks(range(len(N_values)), N_values)
plt.legend()
plt.grid(alpha=0.5, linestyle='--')
plt.title('Surface Energy vs N for Different Ranges')
plt.show()

chosen_key = input("Enter the key of the chosen range: ")

plt.figure(figsize=(8, 6))
chosen_surface_energy = surface_energy_dict[chosen_key]
print(chosen_surface_energy)
plt.plot(range(len(N_values)), chosen_surface_energy, marker='o', label="Fiorentini Method")
plt.xlabel('Number of Layers')
plt.ylabel('Surface Energy (J $m^{-2}$)')
plt.xticks(range(len(N_values)), N_values)
plt.legend()
plt.grid(alpha=0.7, linestyle='--')
plt.savefig("Me_3.440-in-ev.pdf", dpi=700)
plt.show()


In [ ]:
#-------------------------| Solvated Surface Energy |--------------------------------#
#------------------------------| In eV/ang^2 |---------------------------------------#
import re
import numpy as np
import pandas as pd
from ase.io import read
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import linregress

def get_energy_of_pure_electrolyte(filename):
    lines = Path(filename).read_text().splitlines()
    pe = [line for line in lines if 'Energy of pure electrolyte:' in line]
    return float(pe[0].split()[6])

def get_accessible_fraction(filename):
    lines = Path(filename).read_text().splitlines()
    af = [i for i in lines if re.findall(r'Accessible fraction:', i)]
    return float(re.findall(r'Accessible fraction:\s+(\d+\.\d+)', af[0])[0])

parent_directory = Path("/Users/bdayers/Downloads/Surface-energy")
N_values = np.array([1, 3, 5, 7, 9, 11, 13, 15])
Eslab_values = []
A_value = None
ha_to_ev = 27.211386245988

for subdirectory in parent_directory.rglob("*-thick"):
    if subdirectory.is_dir():
        out_file = next(subdirectory.glob("Li.out"), None)
        if out_file:
            atoms = read(str(out_file))
            energy = atoms.get_potential_energy()
            Eslab_values.append(energy)
            energy_of_pure_electrolyte_value = get_energy_of_pure_electrolyte(str(out_file)) 
            accessible_fraction_value = get_accessible_fraction(str(out_file)) 
            Epe = accessible_fraction_value * energy_of_pure_electrolyte_value * ha_to_ev
            print(f"Result for {subdirectory.name}: {Epe}")
            if A_value is None:
                A_value = np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1])) 
                
Eslab_values = np.array(Eslab_values)
Eslab_values = np.sort(Eslab_values)[::-1]
surface_energy_dict = {}
plt.figure(figsize=(8, 6))

for i in range(len(N_values) - 1):
    slope, _, _, _, _ = linregress(N_values[i:], Eslab_values[i:])
    surface_energy = (1 / (2 * A_value)) * ((-N_values * slope) + (Eslab_values - Epe))
    key = f"{N_values[i]} to {N_values[-1]}"
    surface_energy_dict[key] = surface_energy
    plt.plot(range(len(N_values)), surface_energy, marker='o', label=key)

df = pd.DataFrame(surface_energy_dict, index=N_values)
df.index.name = 'Number of Layers'
df.columns.name = 'Surface Energy (eV $Å^{-2}$)'

plt.legend()
plt.xlabel('Number of Layers')
plt.grid(alpha=0.5, linestyle='--')
plt.xticks(range(len(N_values)), N_values)
plt.ylabel('Surface Energy (eV $Å^{-2}$)')
plt.title('Surface Energy vs N for Different Ranges')
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter("{x:.4f}"))
plt.show()

chosen_key = input("Enter the key of the chosen range: ")
chosen_surface_energy = df.loc[:, chosen_key]

plt.figure(figsize=(8, 6))
print(chosen_surface_energy)
plt.plot(range(len(N_values)), chosen_surface_energy, marker='o', label="Fiorentini Method")
plt.xlabel('Number of Layers')
plt.ylabel('Surface Energy (eV $Å^{-2}$)')
plt.xticks(range(len(N_values)), N_values)
plt.legend()
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter("{x:.4f}"))
plt.grid(alpha=0.7, linestyle='--')
plt.savefig("Me_3.440-in-ev.pdf", dpi=700)
plt.show()


In [ ]:
#-------------------------| Solvated Surface Energy |--------------------------------#
#--------------------------------| In J/m^2 |----------------------------------------#
import re
import numpy as np
import pandas as pd
from ase.io import read
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import linregress

def get_energy_of_pure_electrolyte(filename):
    lines = Path(filename).read_text().splitlines()
    pe = [line for line in lines if 'Energy of pure electrolyte:' in line]
    return float(pe[0].split()[6])

def get_accessible_fraction(filename):
    lines = Path(filename).read_text().splitlines()
    af = [i for i in lines if re.findall(r'Accessible fraction:', i)]
    return float(re.findall(r'Accessible fraction:\s+(\d+\.\d+)', af[0])[0])

parent_directory = Path("/Users/bdayers/Downloads/Surface-energy")
N_values = np.array([1, 3, 5, 7, 9, 11, 13, 15])
Eslab_values = []
A_value = None
ha_to_joules = 4.3597447222071E-18
ev_to_joules = 1.602176634E-19

for subdirectory in parent_directory.rglob("*-thick"):
    if subdirectory.is_dir():
        out_file = next(subdirectory.glob("Li.out"), None)
        if out_file:
            atoms = read(str(out_file))
            energy = atoms.get_potential_energy() * ev_to_joules 
            Eslab_values.append(energy)
            energy_of_pure_electrolyte_value = get_energy_of_pure_electrolyte(str(out_file)) 
            accessible_fraction_value = get_accessible_fraction(str(out_file)) 
            Epe = accessible_fraction_value * energy_of_pure_electrolyte_value * ha_to_joules
            print(f"Result for {subdirectory.name}: {Epe}")
            if A_value is None:
                A_value = np.linalg.norm(np.cross(atoms.cell[0], atoms.cell[1])) * 1E-20
                
Eslab_values = np.array(Eslab_values)
Eslab_values = np.sort(Eslab_values)[::-1]
surface_energy_dict = {}
plt.figure(figsize=(8, 6))

for i in range(len(N_values) - 1):
    slope, _, _, _, _ = linregress(N_values[i:], Eslab_values[i:])
    surface_energy = (1 / (2 * A_value)) * ((-N_values * slope) + (Eslab_values + Epe))
    key = f"{N_values[i]} to {N_values[-1]}"
    surface_energy_dict[key] = surface_energy
    plt.plot(range(len(N_values)), surface_energy, marker='o', label=key)

df = pd.DataFrame(surface_energy_dict, index=N_values)
df.index.name = 'Number of Layers'
df.columns.name = 'Surface Energy (J $m^{-2}$)'

plt.legend()
plt.xlabel('Number of Layers')
plt.grid(alpha=0.5, linestyle='--')
plt.xticks(range(len(N_values)), N_values)
plt.ylabel('Surface Energy (J $m^{-2}$)')
plt.title('Surface Energy vs N for Different Ranges')
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter("{x:.3f}"))
plt.show()

chosen_key = input("Enter the key of the chosen range: ")
chosen_surface_energy = df.loc[:, chosen_key]
average_FM = sum(chosen_surface_energy[-4:])/4
print(f'Averaged surface energy {average_FM:.3f} J $m^{-2}$')

plt.figure(figsize=(8, 6))
print(chosen_surface_energy)
plt.plot(range(len(N_values)), chosen_surface_energy, marker='o', label="Fiorentini Method")
plt.xlabel('Number of Layers')
plt.ylabel('Surface Energy (J $m^{-2}$)')
plt.xticks(range(len(N_values)), N_values)
plt.legend()
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter("{x:.3f}"))
plt.grid(alpha=0.7, linestyle='--')
plt.savefig("Me_3.440-in-ev.pdf", dpi=700)
plt.show()
